# Conversation History with LangChain

Understand how LLMs handle (and lose) conversation context, and how to manage it manually.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand LLM statelessness** - LLMs have no memory; each call is independent
2. **Use LangChain message types** - `HumanMessage`, `AIMessage`, and `SystemMessage`
3. **Manage conversation history** - Maintain context across multiple turns using a message list

## 1. Environment Setup

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

print("✅ All imports successful")

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

print("✅ LLM initialized")

## 2. LLMs Are Stateless

Each LLM call is completely independent. The model has **no memory** of previous calls.

Let's prove this with a simple example.

In [ ]:
# Call 1: Tell the LLM something
response1 = llm.invoke([HumanMessage(content="My name is Ravi and I live in Hyderabad.")])
print(f"Call 1 Response:\n{response1.content}")

In [ ]:
# Call 2: Ask about what we just said
response2 = llm.invoke([HumanMessage(content="What is my name and where do I live?")])
print(f"Call 2 Response:\n{response2.content}")

The LLM **does not remember** Call 1 when processing Call 2. Each call starts fresh with no context.

This is a fundamental property of LLMs — they are stateless functions that only see what you pass in the current request.

## 3. LangChain Message Types

LangChain provides three core message types to structure conversations:

| Message Type | Purpose | Example |
|---|---|---|
| `SystemMessage` | Sets the LLM's behavior/role | "You are a helpful assistant" |
| `HumanMessage` | User input | "What is the capital of France?" |
| `AIMessage` | LLM's response | "The capital of France is Paris." |

In [ ]:
# Create each message type
system_msg = SystemMessage(content="You are a helpful travel assistant.")
human_msg = HumanMessage(content="What is the capital of France?")
ai_msg = AIMessage(content="The capital of France is Paris.")

In [ ]:
# Inspect the message structure
for msg in [system_msg, human_msg, ai_msg]:
    print(f"Type: {msg.type:10s} | Content: {msg.content}")

## 4. Passing Conversation History Manually

The solution to statelessness is simple: **pass the entire conversation history** with every call.

The LLM accepts a list of messages, so we include all previous messages in each request.

In [ ]:
# Build the full conversation and pass it
messages = [
    HumanMessage(content="My name is Ravi and I live in Hyderabad."),
    AIMessage(content="Nice to meet you, Ravi! Hyderabad is a wonderful city."),
    HumanMessage(content="What is my name and where do I live?")
]

response = llm.invoke(messages)
print(f"Response:\n{response.content}")

Now the LLM **remembers** because we included the full conversation in the request.

The LLM didn't actually "remember" anything — it simply read all the messages we passed and responded based on the full context.

## 5. Building a Multi-Turn Conversation

Let's build a proper multi-turn conversation by accumulating messages in a Python list.

In [ ]:
# Start with a system message
conversation = [
    SystemMessage(content="You are a Toyota car sales assistant. Keep responses brief and helpful.")
]

In [ ]:
# Turn 1
conversation.append(HumanMessage(content="I'm looking for a family SUV. What do you recommend?"))

response = llm.invoke(conversation)
conversation.append(response)

print(f"Turn 1 ({len(conversation)} messages):")
print(response.content)

In [ ]:
# Turn 2 - Reference previous context
conversation.append(HumanMessage(content="What's the price of the first one you mentioned?"))

response = llm.invoke(conversation)
conversation.append(response)

print(f"Turn 2 ({len(conversation)} messages):")
print(response.content)

In [ ]:
# Turn 3 - Use pronoun referencing
conversation.append(HumanMessage(content="Does it come in a hybrid version?"))

response = llm.invoke(conversation)
conversation.append(response)

print(f"Turn 3 ({len(conversation)} messages):")
print(response.content)

The LLM correctly resolves:
- **"the first one"** → refers to the SUV from Turn 1
- **"it"** → refers to the car being discussed in Turn 2

This works because we pass **all previous messages** on every call.

## 6. Inspecting the Conversation State

Let's look at what our conversation list contains after 3 turns.

In [ ]:
print(f"Total messages in conversation: {len(conversation)}")
print()

for i, msg in enumerate(conversation):
    label = msg.type.upper()
    preview = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  [{i}] {label:8s} | {preview}")

**Pattern:** Each turn adds 2 messages (1 Human + 1 AI), so the list grows as:

```
Start:  1 message  (SystemMessage)
Turn 1: 3 messages (+1 Human, +1 AI)
Turn 2: 5 messages (+1 Human, +1 AI)
Turn 3: 7 messages (+1 Human, +1 AI)
```

This growing list **is** the conversation history.

## 7. The Challenge of Manual Management

Managing conversation history manually works, but has drawbacks:

1. **You must remember to append** every Human and AI message
2. **The list grows indefinitely** — no built-in size management
3. **Tool messages add complexity** — when tools are involved, you need to track `ToolMessage` responses too
4. **Easy to make mistakes** — forgetting to append a message breaks the context

In the next notebook, we'll see how **LangGraph's `MessagesState`** solves these problems by automatically managing message accumulation within a graph workflow.

## Conclusion

### What You've Accomplished

✅ **Understood LLM statelessness** — Each LLM call is independent with no built-in memory

✅ **Used LangChain message types** — `HumanMessage`, `AIMessage`, and `SystemMessage` to structure conversations

✅ **Managed conversation history** — Built multi-turn conversations by accumulating messages in a list

### Key Insight

**LLMs don't remember — you pass the memory.** Every call requires the full conversation history to maintain context. This is the fundamental pattern that all conversation systems (including LangGraph) build upon.

### Next Steps

Continue to **Notebook: Graph Construction** to learn how LangGraph's `MessagesState` automates this message management within a graph workflow.